<a href="https://colab.research.google.com/github/prabal4546/User-perception-analysis-of-cyber-attack-consequences-using-NLP/blob/main/PA2_2_Shah_Siddiqui_Walia_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk import pos_tag

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import wordnet



# Dataset Loading and Preprocessing

In [ ]:
PATH = os.path.join("/content/sample_data/")
dataset = os.path.join("all_data.xlsx")
def load_dataset(path = PATH, dataset = []):
  excel_path = os.path.join(path, dataset)
  return pd.read_excel(excel_path, sheet_name="data formatted for R")

raw_dataset = load_dataset(PATH, dataset)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
raw_dataset

,The cyber-attacker accessed your computer files.,The cyber-attacker accessed your computer programs.,The cyber-attacker accessed your information stored in an Internet site.,The cyber-attacker caused a program on your computer to crash.,The cyber-attacker caused your computer program to run very slowly.,The cyber-attacker caused your computer to crash.,The cyber-attacker caused your computer to run very slowly.,The cyber-attacker caused your Internet connection to run very slowly.,The cyber-attacker caused your request for a certain Internet page to actually take you to a different Internet page.,The cyber-attacker changed a device’s serial number.,...,The cyber-attacker prevented you from logging into your computer.,The cyber-attacker prevented you from using your computer until you pay a ransom.,The cyber-attacker removed your computer files in order to hide their activities.,The cyber-attacker rerouted your Internet requests to a device that they control.,The cyber-attacker saw what was presented on your computer screen.,The cyber-attacker sent you an email that asks you to click on a given Internet link.,The cyber-attacker sent you an email that asks you to respond with certain personal information.,The cyber-attacker shut down an Internet site that you were using.,The cyber-attacker took control over one of your financial accounts.,The cyber-attacker used your computer to store and distribute stolen software.
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,0,1,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
223,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
224,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1


In [ ]:
df = raw_dataset.reset_index()
col= list(df.columns)
descriptions = []
for index, row in df.iterrows():
  description = ""
  for i in range(len(row)):
    if row[i]==1:
      if not description:
        description = description + col[i]
      else:
        description = description + " " + col[i]
  descriptions.append(description)
df['Descriptions'] = descriptions

In [ ]:
df2 = df.filter(['Descriptions'], axis=1)
df2

,Descriptions
0,The cyber-attacker determined your password. T...
1,index The cyber-attacker floods your inbox wit...
2,The cyber-attacker gained information about th...
3,The cyber-attacker changed how an Internet ser...
4,The cyber-attacker made your computer run soft...
...,...
221,The cyber-attacker accessed your computer prog...
222,The cyber-attacker floods your inbox with a ve...
223,The cyber-attacker caused your computer progra...
224,The cyber-attacker accessed your computer file...


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

# Initialize stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text, remove_stopwords=True, apply_stemming=True):
    # Tokenize
    tokens = word_tokenize(text.lower())

    # Remove stop words
    if remove_stopwords:
        tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    if apply_stemming:
        tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)

documents = df2['Descriptions'].tolist()



preprocessed_documents = [preprocess_text(doc) for doc in documents]

preprocessed_documents[:5]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['cyber-attack determin password . cyber-attack gain usernam password given internet site . cyber-attack made comput think password enter .',
 'index cyber-attack flood inbox larg number email . cyber-attack made think email receiv attack came someon els . cyber-attack sent email ask click given internet link . cyber-attack sent email ask respond certain person inform .',
 'cyber-attack gain inform devic use creat home network . cyber-attack prevent access home network .',
 'cyber-attack chang internet servic function benefit attack . cyber-attack made comput perform task benefit attack .',
 'cyber-attack made comput run softwar comput intend run . cyber-attack modifi inform store internet site . cyber-attack modifi inform within internet databas . cyber-attack rerout internet request devic control .']

In [ ]:
documents_with_stopwords = [preprocess_text(doc, remove_stopwords=False) for doc in documents]

# Vectorization

In [ ]:
# BOW Vectorization
bow_vectorizer = CountVectorizer(ngram_range=(1,1)) # Using unigrams for now
bow_matrix = bow_vectorizer.fit_transform(preprocessed_documents)

bow_similarity_matrix = cosine_similarity(bow_matrix)

bow_similarity_matrix[:5, :5]

array([[1.        , 0.5560594 , 0.44563426, 0.6103001 , 0.61802547],
       [0.5560594 , 1.        , 0.43226497, 0.57863756, 0.55571893],
       [0.44563426, 0.43226497, 1.        , 0.44232587, 0.48043565],
       [0.6103001 , 0.57863756, 0.44232587, 1.        , 0.64311969],
       [0.61802547, 0.55571893, 0.48043565, 0.64311969, 1.        ]])

In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1)) # Using unigrams for now
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_documents)

tfidf_similarity_matrix = cosine_similarity(tfidf_matrix)

tfidf_similarity_matrix[:5, :5]

array([[1.        , 0.21482022, 0.1827419 , 0.24946864, 0.27586866],
       [0.21482022, 1.        , 0.12920478, 0.19823781, 0.21216366],
       [0.1827419 , 0.12920478, 1.        , 0.14811858, 0.22383041],
       [0.24946864, 0.19823781, 0.14811858, 1.        , 0.29806668],
       [0.27586866, 0.21216366, 0.22383041, 0.29806668, 1.        ]])

In [ ]:
configs = [
    {"remove_stopwords": True, "apply_stemming": True, "ngram_range": (1,1)},
    {"remove_stopwords": True, "apply_stemming": True, "ngram_range": (1,2)},
    {"remove_stopwords": True, "apply_stemming": True, "ngram_range": (1,3)},
    {"remove_stopwords": True, "apply_stemming": False, "ngram_range": (1,1)},
    {"remove_stopwords": True, "apply_stemming": False, "ngram_range": (1,2)},
    {"remove_stopwords": True, "apply_stemming": False, "ngram_range": (1,3)},
    {"remove_stopwords": False, "apply_stemming": True, "ngram_range": (1,1)},
    {"remove_stopwords": False, "apply_stemming": True, "ngram_range": (1,2)},
    {"remove_stopwords": False, "apply_stemming": True, "ngram_range": (1,3)},
    {"remove_stopwords": False, "apply_stemming": False, "ngram_range": (1,1)},
    {"remove_stopwords": False, "apply_stemming": False, "ngram_range": (1,2)},
    {"remove_stopwords": False, "apply_stemming": False, "ngram_range": (1,3)}
]

similarity_matrices = {
    "BOW": [],
    "TF-IDF": []
}

# Process each configuration
for config in configs:
    config_documents = [preprocess_text(doc, config["remove_stopwords"], config["apply_stemming"]) for doc in documents]

    # BOW Vectorization
    bow_vectorizer = CountVectorizer(ngram_range=config["ngram_range"])
    bow_matrix = bow_vectorizer.fit_transform(config_documents)
    bow_similarity_matrix = cosine_similarity(bow_matrix)
    similarity_matrices["BOW"].append(bow_similarity_matrix)

    # TF-IDF Vectorization
    tfidf_vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"])
    tfidf_matrix = tfidf_vectorizer.fit_transform(config_documents)
    tfidf_similarity_matrix = cosine_similarity(tfidf_matrix)
    similarity_matrices["TF-IDF"].append(tfidf_similarity_matrix)

(len(similarity_matrices["BOW"]), similarity_matrices["BOW"][0].shape, len(similarity_matrices["TF-IDF"]), similarity_matrices["TF-IDF"][0].shape)


(12, (226, 226), 12, (226, 226))

# Spearman's rank correlation

In [ ]:
from scipy.stats import spearmanr

vectorized_data = raw_dataset.iloc[:, 2:52]

spearman_correlation_matrix, _ = spearmanr(vectorized_data, axis=1)

spearman_correlation_df = pd.DataFrame(spearman_correlation_matrix)

spearman_correlation_df.shape, spearman_correlation_df.head()


((226, 226),
         0         1         2         3         4         5         6    \
 0  1.000000 -0.077850 -0.053838 -0.053838 -0.077850 -0.132453 -0.097590   
 1 -0.077850  1.000000 -0.062869 -0.062869 -0.090909 -0.154672 -0.113961   
 2 -0.053838 -0.062869  1.000000 -0.043478 -0.062869 -0.106966 -0.078811   
 3 -0.053838 -0.062869 -0.043478  1.000000 -0.062869 -0.106966 -0.078811   
 4 -0.077850 -0.090909 -0.062869 -0.062869  1.000000 -0.154672 -0.113961   
 
         7         8         9    ...       216       217       218       219  \
 0 -0.066667 -0.097590 -0.097590  ... -0.088045 -0.097590 -0.088045 -0.053838   
 1 -0.077850 -0.113961 -0.113961  ... -0.102815 -0.113961 -0.102815 -0.062869   
 2 -0.053838 -0.078811 -0.078811  ... -0.071103 -0.078811 -0.071103 -0.043478   
 3 -0.053838 -0.078811 -0.078811  ...  0.270191 -0.078811  0.270191 -0.043478   
 4 -0.077850 -0.113961 -0.113961  ...  0.143940 -0.113961  0.143940 -0.062869   
 
         220       221       222       22

In [ ]:

from scipy.stats import pearsonr

# Function to extract the upper triangle of a matrix without the diagonal
def get_upper_triangle(matrix):
    return matrix[np.triu_indices_from(matrix, k=1)]

# Extract the upper triangle of the Spearman's matrix
spearman_upper_triangle = get_upper_triangle(spearman_correlation_matrix)

# Initialize an empty list to store Pearson correlation coefficients
pearson_correlations = []

for matrix_type, matrices in similarity_matrices.items():
    for i, matrix in enumerate(matrices):
        upper_triangle = get_upper_triangle(matrix)

        # Calculate the Pearson correlation with Spearman's upper triangle
        correlation, _ = pearsonr(spearman_upper_triangle, upper_triangle)

        # Store the correlation coefficient with details about the configuration
        pearson_correlations.append({
            "type": matrix_type,
            "config_index": i,
            "pearson_correlation": correlation
        })

# Convert the list of correlation results to a DataFrame for better display and analysis
pearson_correlation_df = pd.DataFrame(pearson_correlations)

# Display the sorted correlation results
pearson_correlation_df.sort_values(by='pearson_correlation', ascending=False)


,type,config_index,pearson_correlation
17,TF-IDF,5,0.784768
14,TF-IDF,2,0.781708
23,TF-IDF,11,0.740193
20,TF-IDF,8,0.737662
16,TF-IDF,4,0.736207
13,TF-IDF,1,0.730810
22,TF-IDF,10,0.691693
19,TF-IDF,7,0.687516
15,TF-IDF,3,0.670197
12,TF-IDF,0,0.657672


# Word Net

In [ ]:
# Tokenize and apply POS tagging

nltk.download('averaged_perceptron_tagger')
tagged_descriptions = [pos_tag(word_tokenize(description)) for description in df2['Descriptions'].astype(str)]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:

df2['tagged_descriptions'] = df2['Descriptions'].astype(str).apply(lambda x: pos_tag(word_tokenize(x)))

print(df2[['Descriptions', 'tagged_descriptions']])


                                          Descriptions  \
0    The cyber-attacker determined your password. T...   
1    index The cyber-attacker floods your inbox wit...   
2    The cyber-attacker gained information about th...   
3    The cyber-attacker changed how an Internet ser...   
4    The cyber-attacker made your computer run soft...   
..                                                 ...   
221  The cyber-attacker accessed your computer prog...   
222  The cyber-attacker floods your inbox with a ve...   
223  The cyber-attacker caused your computer progra...   
224  The cyber-attacker accessed your computer file...   
225  The cyber-attacker gained information about ex...   

                                   tagged_descriptions  
0    [(The, DT), (cyber-attacker, NN), (determined,...  
1    [(index, NN), (The, DT), (cyber-attacker, NN),...  
2    [(The, DT), (cyber-attacker, NN), (gained, VBN...  
3    [(The, DT), (cyber-attacker, NN), (changed, VB...  
4    [(The, DT), (

In [ ]:
nltk.download('wordnet')

# Function to map NLTK POS tags to WordNet POS tags
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

example_tagged_description = tagged_descriptions[0]  # Assuming this is your tagged descriptions from the previous step
wordnet_tagged_description = [(word, nltk_tag_to_wordnet_tag(tag)) for word, tag in example_tagged_description]

wordnet_tagged_description

[nltk_data] Downloading package wordnet to /root/nltk_data...


[('The', None),
 ('cyber-attacker', 'n'),
 ('determined', 'v'),
 ('your', None),
 ('password', 'n'),
 ('.', None),
 ('The', None),
 ('cyber-attacker', 'n'),
 ('gains', 'n'),
 ('your', None),
 ('username', 'n'),
 ('and', None),
 ('password', 'n'),
 ('for', None),
 ('a', None),
 ('given', 'v'),
 ('Internet', 'n'),
 ('site', 'n'),
 ('.', None),
 ('The', None),
 ('cyber-attacker', 'n'),
 ('made', 'v'),
 ('your', None),
 ('computer', 'n'),
 ('think', 'v'),
 ('that', None),
 ('your', None),
 ('password', 'n'),
 ('was', 'v'),
 ('entered', 'v'),
 ('when', None),
 ('it', None),
 ('was', 'v'),
 ('not', 'r'),
 ('.', None)]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()

# Generate document vectors
tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)

# Calculate the pairwise similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)


In [ ]:
similarity_matrix[:5]

array([[1.        , 0.21482022, 0.1827419 , ..., 0.19376876, 0.5673138 ,
        0.23836349],
       [0.21482022, 1.        , 0.12920478, ..., 0.13972002, 0.26799611,
        0.11266153],
       [0.1827419 , 0.12920478, 1.        , ..., 0.1188624 , 0.44661354,
        0.14977301],
       [0.24946864, 0.19823781, 0.14811858, ..., 0.22426399, 0.38371227,
        0.14083436],
       [0.27586866, 0.21216366, 0.22383041, ..., 0.40871102, 0.50324843,
        0.2013311 ]])

In [ ]:
processed_texts_without_stopwords = preprocessed_documents
vectorizer_without_sw = TfidfVectorizer()
tfidf_matrix_without_sw = vectorizer_without_sw.fit_transform(processed_texts_without_stopwords)

similarity_matrix_without_sw = cosine_similarity(tfidf_matrix_without_sw)

In [ ]:
print("Cosine Similarity Matrix without Stop Words:")
print(similarity_matrix_without_sw)

Cosine Similarity Matrix without Stop Words:
[[1.         0.21482022 0.1827419  ... 0.19376876 0.5673138  0.23836349]
 [0.21482022 1.         0.12920478 ... 0.13972002 0.26799611 0.11266153]
 [0.1827419  0.12920478 1.         ... 0.1188624  0.44661354 0.14977301]
 ...
 [0.19376876 0.13972002 0.1188624  ... 1.         0.32178408 0.10739416]
 [0.5673138  0.26799611 0.44661354 ... 0.32178408 1.         0.28582874]
 [0.23836349 0.11266153 0.14977301 ... 0.10739416 0.28582874 1.        ]]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity matrices for both configurations
wordnet_vectors_without_sw = tfidf_vectorizer.fit_transform(raw_dataset)
similarity_matrix_without_sw = cosine_similarity(wordnet_vectors_without_sw)

print("4x1 Similarity Matrix with Stop Words:")
print(similarity_matrix_without_sw.reshape(-1, 1))


4x1 Similarity Matrix with Stop Words:
[[1.        ]
 [0.67064344]
 [0.39092583]
 ...
 [0.01543845]
 [0.01890159]
 [1.        ]]


In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Greedy Lemma Overlap

In [ ]:
from nltk.corpus import wordnet as wn
from itertools import product
from nltk.tokenize import word_tokenize

def greedy_lemma_aligning_overlap_debug(text1, text2):
    synsets1 = [wn.synsets(word) for word in text1]
    synsets2 = [wn.synsets(word) for word in text2]

    print(f"Synsets for the first word in text1: {synsets1[0]}")
    print(f"Synsets for the first word in text2: {synsets2[0]}")

    # Filter out empty synset lists
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]

    print(f"Filtered synsets for text1: {synsets1[:5]}")
    print(f"Filtered synsets for text2: {synsets2[:5]}")

    if not synsets1 or not synsets2:
        return 0

    max_similarities = []
    for synset_group1 in synsets1:
        max_similarity = 0
        for synset1 in synset_group1:
            for synset_group2 in synsets2:
                for synset2 in synset_group2:
                    similarity = synset1.path_similarity(synset2)
                    if similarity and similarity > max_similarity:
                        max_similarity = similarity
        max_similarities.append(max_similarity)

    similarity_score = max_similarity/max(len(text1), len(text2))
    return similarity_score

In [ ]:
sentence1 = description[0][0]
sentence2 = tagged_descriptions[1][0]

for synset_pair in product (*[sentence1, sentence2]):
        for syn1, syn2 in product(*synset_pair):
            similarity = greedy_lemma_aligning_overlap_debug(sentence1, sentence2)

print (f"The Greedy Lemma Aligning Overlap similarity is: {similarity}")

Synsets for the first word in text1: [Synset('thymine.n.01'), Synset('deoxythymidine_monophosphate.n.01'), Synset('metric_ton.n.01'), Synset('t.n.04'), Synset('triiodothyronine.n.01'), Synset('thyroxine.n.01')]
Synsets for the first word in text2: [Synset('index.n.01'), Synset('index.n.02'), Synset('exponent.n.03'), Synset('index.n.04'), Synset('index.n.05'), Synset('index.v.01'), Synset('index.v.02'), Synset('index.v.03')]
Filtered synsets for text1: [[Synset('thymine.n.01'), Synset('deoxythymidine_monophosphate.n.01'), Synset('metric_ton.n.01'), Synset('t.n.04'), Synset('triiodothyronine.n.01'), Synset('thyroxine.n.01')]]
Filtered synsets for text2: [[Synset('index.n.01'), Synset('index.n.02'), Synset('exponent.n.03'), Synset('index.n.04'), Synset('index.n.05'), Synset('index.v.01'), Synset('index.v.02'), Synset('index.v.03')]]
Synsets for the first word in text1: [Synset('thymine.n.01'), Synset('deoxythymidine_monophosphate.n.01'), Synset('metric_ton.n.01'), Synset('t.n.04'), Synset